In [16]:
import keyring
from pprint import pprint

api_key_irp = keyring.get_password("irp_api_key", "birdblues")
api_secret_irp = keyring.get_password("irp_api_secret", "birdblues")

api_key_isa = keyring.get_password("isa_api_key", "birdblues")
api_secret_isa = keyring.get_password("isa_api_secret", "birdblues")

api_key_0 = keyring.get_password("api_key_0", "birdblues")
api_secret_0 = keyring.get_password("api_secret_0","birdblues")

api_key_1 = keyring.get_password("api_key_1", "birdblues")
api_secret_1 = keyring.get_password("api_secret_1","birdblues")

api_key_2 = keyring.get_password("api_key_2", "birdblues")
api_secret_2 = keyring.get_password("api_secret_2","birdblues")

# print(f"api_key_irp = {api_key_irp}")
# print(f"api_secret_irp = {api_secret_irp}")
# print(f"api_key_isa = {api_key_isa}")
# print(f"api_secret_isa = {api_secret_isa}")
# print(f"api_key_0 = {api_key_0}")
# print(f"api_secret_0 = {api_secret_0}")
# print(f"api_key_1 = {api_key_1}")
# print(f"api_secret_1 = {api_secret_1}")
# print(f"api_key_2 = {api_key_2}")
# print(f"api_secret_2 = {api_secret_2}")

In [17]:
acc_no_0 = "63981494-01" # 위탁
acc_no_1 = "64267316-22" # 연금저축 1
acc_no_2 = "64012548-22" # 연금저축 2
acc_no_isa = "64043278-01" # ISA
acc_no_irp = "64012548-29" # IR

In [18]:
import keyring

token_0 = keyring.get_password("access_token_0", "birdblues")
token_1 = keyring.get_password("access_token_1", "birdblues")
token_2 = keyring.get_password("access_token_2", "birdblues")
token_3 = keyring.get_password("access_token_3", "birdblues")
token_4 = keyring.get_password("access_token_4", "birdblues")

# print(f"token_0 = {token_0}")
# print(f"token_1 = {token_1}")
# print(f"token_2 = {token_2}")
# print(f"token_3 = {token_3}")
# print(f"token_4 = {token_4}")

In [19]:
import requests
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

column_mapping = {
    'trad_dt': '거래일',
    'pdno': '코드',
    'prdt_name': '상품명',
    'trad_dvsn_name': '거래구분',
    'loan_dt': '대출일',
    'hldg_qty': '잔고',
    'pchs_unpr': '매입가격',
    'buy_qty': '매수수량',
    'buy_amt': '매수금액',
    'sll_pric': '매도가격',
    'sll_qty': '매도수량',
    'sll_amt': '매도금액',
    'rlzt_pfls': '실현손익',
    'pfls_rt': '손익률',
    'fee': '수수료',
    'tl_tax': '세금',
    'loan_int': '대출이자'
}

cols_to_convert = ['잔고', '매입가격', '매수수량', '매수금액', '매도가격', '매도수량', '매도금액', '실현손익', '손익률', '수수료', '세금', '대출이자']

def get_trade_profit(access_token, app_key, app_secret, acc_no):
    
    output1_list = []
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/inquire-period-trade-profit"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC8715R"
        # "tr_cont": "",
        # "custtype": "P"
        
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "SORT_DVSN": "01",
        "PDNO": "",
        "INQR_STRT_DT": "20230101",
        "INQR_END_DT": "20241231",
        "CBLC_DVSN": "00",
        "CTX_AREA_NK100": "",
        "CTX_AREA_FK100": ""
    }
    res = requests.get(url, headers=headers, params=params)
    body = res.json()
    if 'output1' in body:
        output1_list.extend(body['output1'])
        
    # display(json)
    # print(res.headers['tr_cont'])

    while res.headers['tr_cont'] == "M":
        headers["tr_cont"] = res.headers['tr_cont']
        params["INQR_STRT_DT"] = body['ctx_area_nk100'][:8]
        params["CTX_AREA_NK100"] = body['ctx_area_nk100']
        params["CTX_AREA_FK100"] = body['ctx_area_fk100']
        res = requests.get(url, headers=headers, params=params)
        body = res.json()
        if 'output1' in body:
            output1_list.extend(body['output1'])
        # display(json)
        # print(res.headers['tr_cont'])
    df = pd.DataFrame(output1_list)
    df.rename(columns=column_mapping, inplace=True)
 
    # 각 컬럼에 대해서 쉼표 제거하고 숫자형으로 변환
    for col in cols_to_convert:
        df[col] = df[col].str.replace(',', '')  # 쉼표 제거
        df[col] = pd.to_numeric(df[col], errors='coerce')  # 숫자형으로 변환 (에러가 발생하면 NaN으로 처리)
        
    df['계좌'] = acc_no
    df.loc[df['매도수량'] > 0, '종류'] = '거래소매도'
    df.loc[df['매수수량'] > 0, '종류'] = '거래소매수'
    df.loc[df['매도수량'] > 0, '거래금'] = df['매도금액']
    df.loc[df['매수수량'] > 0, '거래금'] = df['매수금액']
    df.loc[df['매도수량'] > 0, '수량'] = df['매도수량']
    df.loc[df['매수수량'] > 0, '수량'] = df['매수수량']
    df.loc[df['매도수량'] > 0, '정산금'] = df['거래금'] - df['수수료'] - df['세금']
    df.loc[df['매수수량'] > 0, '정산금'] = df['거래금'] + df['수수료'] + df['세금']
    df['거래일'] = pd.to_datetime(df['거래일'], format='%Y%m%d')
    df = df[['거래일', '종류', '거래금', '수량', '정산금', '수수료', '세금', '상품명', '코드', '계좌']]
    df.set_index('거래일', inplace=True)
    df.sort_index(inplace=True)
    
    return df

In [20]:
# access_token_0 = get_access_token(api_key_0, api_secret_0)
# access_token_1 = get_access_token(api_key_1, api_secret_1)
# access_token_2 = get_access_token(api_key_2, api_secret_2)
# access_token_3 = get_access_token(api_key_isa, api_secret_isa)
# access_token_4 = get_access_token(api_key_irp, api_secret_irp)

df1 = get_trade_profit(token_0, api_key_0, api_secret_0, acc_no_0)
df2 = get_trade_profit(token_1, api_key_1, api_secret_1, acc_no_1)
df3 = get_trade_profit(token_2, api_key_2, api_secret_2, acc_no_2)
df4 = get_trade_profit(token_3, api_key_isa, api_secret_isa, acc_no_isa)
df5 = get_trade_profit(token_4, api_key_irp, api_secret_irp, acc_no_irp)

df = pd.concat([df1, df2, df3, df4, df5])
df.sort_index(inplace=True)
df

,종류,거래금,수량,정산금,수수료,세금,상품명,코드,계좌
거래일,,,,,,,,,
2023-12-22,거래소매수,114685.00,1.00,114685.00,0,0,KOSEF 국고채10년,148070,64012548-29
2023-12-22,거래소매수,53820.00,6.00,53820.00,0,0,PLUS 신흥국MSCI(합성 H),195980,64012548-29
2023-12-22,거래소매수,106190.00,2.00,106190.00,0,0,TIGER 단기채권액티브,272580,64012548-29
2023-12-22,거래소매수,286580.00,23.00,286580.00,0,0,KODEX 200미국채혼합,284430,64012548-29
2023-12-22,거래소매수,169785.00,11.00,169785.00,0,0,TIGER 미국S&P500,360750,64012548-29
...,...,...,...,...,...,...,...,...,...
2024-09-05,거래소매수,90945.00,9.00,90945.00,0,0,TIGER Fn반도체TOP10,396500,64012548-29
2024-09-05,거래소매수,2005250.00,130.00,2005334.00,84,0,ACE KRX금현물,411060,64012548-22
2024-09-05,거래소매도,2023000.00,200.00,2022915.00,85,0,SOL 미국30년국채커버드콜(합성),473330,64012548-22


In [26]:
# '코드'를 기준으로 데이터를 나누고 MultiIndex 컬럼 생성
def clean_code(df):
    df = df.pivot_table(index=['거래일'], columns='코드', values=['종류', '거래금', '수량', '정산금', '수수료', '세금', '상품명', '계좌'], aggfunc='first')
    df['상품명'] = df['상품명'].apply(lambda col: col.ffill().bfill())
    df['계좌'] = df['계좌'].apply(lambda col: col.ffill().bfill())
    df['종류'] = df['종류'].apply(lambda col: col.ffill().bfill())
    df.fillna(0, inplace=True)

    new_order = ['종류', '거래금', '수량', '정산금', '수수료', '세금', '상품명', '계좌']
    df = df.reorder_levels([1, 0], axis=1).sort_index(axis=1)
    df = df.reindex(columns=pd.MultiIndex.from_product([df.columns.levels[0], new_order]))
    
    return df 

주식 = clean_code(df)
display(주식.columns.levels[0])

for code in 주식.columns.levels[0]:
    display(주식[code])

# 매수 = df[df['종류'] == '거래소매수']
# 매도 = df[df['종류'] == '거래소매도']

# display(매수)

# 결과 확인
df_114800 = 주식.loc[:, ('114800', slice(None))]
display(df_114800)

# dd_114800 = df[df['코드'] == '114800']
# dd_114800

Index(['114800', '148070', '195980', '218420', '272580', '278530', '283580',
       '284430', '329750', '360200', '360750', '365780', '390390', '396500',
       '402970', '411060', '423160', '441680', '446720', '449170', '452360',
       '453810', '456610', '456880', '464540', '468380', '469050', '472870',
       '473330', '475380', '476750', '480020', '482730', '483230', '489250'],
      dtype='object', name='코드')

,종류,거래금,수량,정산금,수수료,세금,상품명,계좌
거래일,,,,,,,,
2023-12-22,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 인버스,64043278-01
2023-12-27,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 인버스,64043278-01
2023-12-28,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 인버스,64043278-01
2024-01-02,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 인버스,64043278-01
2024-01-03,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 인버스,64043278-01
...,...,...,...,...,...,...,...,...
2024-08-23,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 인버스,64043278-01
2024-09-03,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 인버스,64043278-01
2024-09-04,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 인버스,64043278-01


,종류,거래금,수량,정산금,수수료,세금,상품명,계좌
거래일,,,,,,,,
2023-12-22,거래소매수,114685.00,1.00,114685.00,0.00,0.00,KOSEF 국고채10년,64012548-29
2023-12-27,거래소매수,111695.00,1.00,111695.00,0.00,0.00,KOSEF 국고채10년,64012548-29
2023-12-28,거래소매수,224555.00,2.00,224555.00,0.00,0.00,KOSEF 국고채10년,64012548-29
2024-01-02,거래소매수,0.00,0.00,0.00,0.00,0.00,KOSEF 국고채10년,64012548-29
2024-01-03,거래소매수,0.00,0.00,0.00,0.00,0.00,KOSEF 국고채10년,64012548-29
...,...,...,...,...,...,...,...,...
2024-08-23,거래소매도,0.00,0.00,0.00,0.00,0.00,KOSEF 국고채10년,64012548-29
2024-09-03,거래소매도,0.00,0.00,0.00,0.00,0.00,KOSEF 국고채10년,64012548-29
2024-09-04,거래소매도,0.00,0.00,0.00,0.00,0.00,KOSEF 국고채10년,64012548-29


,종류,거래금,수량,정산금,수수료,세금,상품명,계좌
거래일,,,,,,,,
2023-12-22,거래소매수,53820.00,6.00,53820.00,0.00,0.00,PLUS 신흥국MSCI(합성 H),64012548-29
2023-12-27,거래소매수,0.00,0.00,0.00,0.00,0.00,PLUS 신흥국MSCI(합성 H),64012548-29
2023-12-28,거래소매수,64750.00,7.00,64750.00,0.00,0.00,PLUS 신흥국MSCI(합성 H),64012548-29
2024-01-02,거래소매수,0.00,0.00,0.00,0.00,0.00,PLUS 신흥국MSCI(합성 H),64012548-29
2024-01-03,거래소매수,0.00,0.00,0.00,0.00,0.00,PLUS 신흥국MSCI(합성 H),64012548-29
...,...,...,...,...,...,...,...,...
2024-08-23,거래소매도,0.00,0.00,0.00,0.00,0.00,PLUS 신흥국MSCI(합성 H),64012548-29
2024-09-03,거래소매도,0.00,0.00,0.00,0.00,0.00,PLUS 신흥국MSCI(합성 H),64012548-29
2024-09-04,거래소매도,0.00,0.00,0.00,0.00,0.00,PLUS 신흥국MSCI(합성 H),64012548-29


,종류,거래금,수량,정산금,수수료,세금,상품명,계좌
거래일,,,,,,,,
2023-12-22,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국S&P500에너지(합성),64043278-01
2023-12-27,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국S&P500에너지(합성),64043278-01
2023-12-28,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국S&P500에너지(합성),64043278-01
2024-01-02,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국S&P500에너지(합성),64043278-01
2024-01-03,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국S&P500에너지(합성),64043278-01
...,...,...,...,...,...,...,...,...
2024-08-23,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국S&P500에너지(합성),64043278-01
2024-09-03,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국S&P500에너지(합성),64043278-01
2024-09-04,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국S&P500에너지(합성),64043278-01


...

,종류,거래금,수량,정산금,수수료,세금,상품명,계좌
거래일,,,,,,,,
2023-12-22,거래소매수,0.00,0.00,0.00,0.00,0.00,ACE 미국빅테크7+ 15%프리미엄분배(합성),64267316-22
2023-12-27,거래소매수,0.00,0.00,0.00,0.00,0.00,ACE 미국빅테크7+ 15%프리미엄분배(합성),64267316-22
2023-12-28,거래소매수,0.00,0.00,0.00,0.00,0.00,ACE 미국빅테크7+ 15%프리미엄분배(합성),64267316-22
2024-01-02,거래소매수,0.00,0.00,0.00,0.00,0.00,ACE 미국빅테크7+ 15%프리미엄분배(합성),64267316-22
2024-01-03,거래소매수,0.00,0.00,0.00,0.00,0.00,ACE 미국빅테크7+ 15%프리미엄분배(합성),64267316-22
...,...,...,...,...,...,...,...,...
2024-08-23,거래소매수,0.00,0.00,0.00,0.00,0.00,ACE 미국빅테크7+ 15%프리미엄분배(합성),64267316-22
2024-09-03,거래소매도,1575840.00,147.00,1575774.00,66.00,0.00,ACE 미국빅테크7+ 15%프리미엄분배(합성),64267316-22
2024-09-04,거래소매도,0.00,0.00,0.00,0.00,0.00,ACE 미국빅테크7+ 15%프리미엄분배(합성),64267316-22


,종류,거래금,수량,정산금,수수료,세금,상품명,계좌
거래일,,,,,,,,
2023-12-22,거래소매수,0.00,0.00,0.00,0.00,0.00,TIGER 미국S&P500+10%프리미엄초단기,64267316-22
2023-12-27,거래소매수,0.00,0.00,0.00,0.00,0.00,TIGER 미국S&P500+10%프리미엄초단기,64267316-22
2023-12-28,거래소매수,0.00,0.00,0.00,0.00,0.00,TIGER 미국S&P500+10%프리미엄초단기,64267316-22
2024-01-02,거래소매수,0.00,0.00,0.00,0.00,0.00,TIGER 미국S&P500+10%프리미엄초단기,64267316-22
2024-01-03,거래소매수,0.00,0.00,0.00,0.00,0.00,TIGER 미국S&P500+10%프리미엄초단기,64267316-22
...,...,...,...,...,...,...,...,...
2024-08-23,거래소매수,0.00,0.00,0.00,0.00,0.00,TIGER 미국S&P500+10%프리미엄초단기,64267316-22
2024-09-03,거래소매수,0.00,0.00,0.00,0.00,0.00,TIGER 미국S&P500+10%프리미엄초단기,64267316-22
2024-09-04,거래소매수,0.00,0.00,0.00,0.00,0.00,TIGER 미국S&P500+10%프리미엄초단기,64267316-22


,종류,거래금,수량,정산금,수수료,세금,상품명,계좌
거래일,,,,,,,,
2023-12-22,거래소매수,0.00,0.00,0.00,0.00,0.00,RISE 25-06 은행채(AA+이상)액티브,64012548-22
2023-12-27,거래소매수,0.00,0.00,0.00,0.00,0.00,RISE 25-06 은행채(AA+이상)액티브,64012548-22
2023-12-28,거래소매수,0.00,0.00,0.00,0.00,0.00,RISE 25-06 은행채(AA+이상)액티브,64012548-22
2024-01-02,거래소매수,0.00,0.00,0.00,0.00,0.00,RISE 25-06 은행채(AA+이상)액티브,64012548-22
2024-01-03,거래소매수,0.00,0.00,0.00,0.00,0.00,RISE 25-06 은행채(AA+이상)액티브,64012548-22
...,...,...,...,...,...,...,...,...
2024-08-23,거래소매수,0.00,0.00,0.00,0.00,0.00,RISE 25-06 은행채(AA+이상)액티브,64012548-22
2024-09-03,거래소매수,0.00,0.00,0.00,0.00,0.00,RISE 25-06 은행채(AA+이상)액티브,64012548-22
2024-09-04,거래소매수,0.00,0.00,0.00,0.00,0.00,RISE 25-06 은행채(AA+이상)액티브,64012548-22


,종류,거래금,수량,정산금,수수료,세금,상품명,계좌
거래일,,,,,,,,
2023-12-22,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국배당다우존스,64012548-29
2023-12-27,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국배당다우존스,64012548-29
2023-12-28,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국배당다우존스,64012548-29
2024-01-02,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국배당다우존스,64012548-29
2024-01-03,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국배당다우존스,64012548-29
...,...,...,...,...,...,...,...,...
2024-08-23,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국배당다우존스,64012548-29
2024-09-03,거래소매수,392540.00,38.00,392540.00,0.00,0.00,KODEX 미국배당다우존스,64012548-29
2024-09-04,거래소매수,0.00,0.00,0.00,0.00,0.00,KODEX 미국배당다우존스,64012548-29


코드         114800                                                 
               종류  거래금   수량  정산금  수수료   세금        상품명           계좌
거래일                                                               
2023-12-22  거래소매수 0.00 0.00 0.00 0.00 0.00  KODEX 인버스  64043278-01
2023-12-27  거래소매수 0.00 0.00 0.00 0.00 0.00  KODEX 인버스  64043278-01
2023-12-28  거래소매수 0.00 0.00 0.00 0.00 0.00  KODEX 인버스  64043278-01
2024-01-02  거래소매수 0.00 0.00 0.00 0.00 0.00  KODEX 인버스  64043278-01
2024-01-03  거래소매수 0.00 0.00 0.00 0.00 0.00  KODEX 인버스  64043278-01
...           ...  ...  ...  ...  ...  ...        ...          ...
2024-08-23  거래소매수 0.00 0.00 0.00 0.00 0.00  KODEX 인버스  64043278-01
2024-09-03  거래소매수 0.00 0.00 0.00 0.00 0.00  KODEX 인버스  64043278-01
2024-09-04  거래소매수 0.00 0.00 0.00 0.00 0.00  KODEX 인버스  64043278-01
2024-09-05  거래소매수 0.00 0.00 0.00 0.00 0.00  KODEX 인버스  64043278-01
2024-09-09  거래소매수 0.00 0.00 0.00 0.00 0.00  KODEX 인버스  64043278-01

[151 rows x 8 columns]

In [22]:
import yfinance as yf

ticker = '114800.KS' 
stock_data = yf.download(ticker, start="2023-01-01")
close_prices = stock_data['Close']
close_prices = close_prices.resample('D').ffill()
print(close_prices)

[*********************100%%**********************]  1 of 1 completed

Date
2023-01-02   5225.00
2023-01-03   5225.00
2023-01-04   5115.00
2023-01-05   5085.00
2023-01-06   5030.00
               ...  
2024-09-05   4425.00
2024-09-06   4465.00
2024-09-07   4465.00
2024-09-08   4465.00
2024-09-09   4490.00
Freq: D, Name: Close, Length: 617, dtype: float64
